In [1]:
import sqlite3
from astroquery.simbad import Simbad
import numpy as np

In [2]:
connection = sqlite3.connect('nearby_stars.db')

In [3]:
c = connection.cursor()

## Create stars_simbad table

In [433]:
# only run if deleting the table to re-make it:
#c.execute('DROP TABLE IF EXISTS stars_simbad')

In [434]:
create_table = '''
CREATE TABLE IF NOT EXISTS stars_simbad (
id INTEGER PRIMARY KEY AUTOINCREMENT,
main_id TEXT,
hd TEXT,
gj TEXT,
wds TEXT,
ra TEXT,
dec TEXT,
plx REAL,
plx_err REAL,
sptype TEXT,
V_mag REAL,
V_mag_err REAL,
G_mag REAL,
G_mag_err REAL
)
'''

In [435]:
c.execute(create_table)

## Query Simbad for nearby stars

Set maximum distance below.

In [436]:
d_max = 20 #pc
plx_min = 1/(d_max*1e-3)
print 'plx >= {:1.0f} mas for d <= {:1.0f} pc'.format(plx_min, d_max)

plx >= 50 mas for d <= 20 pc


Create custom output fields from Simbad.

In [437]:
customSimbad = Simbad()

In [438]:
customSimbad.reset_votable_fields()
customSimbad.add_votable_fields('id(gj)','id(hd)','id(wds)','plx','plx_error','sp',
                                'flux(V)','flux_error(V)',
                                'flux(G)','flux_error(G)')
customSimbad.get_votable_fields()

['main_id',
 'coordinates',
 'id(gj)',
 'id(hd)',
 'id(wds)',
 'plx',
 'plx_error',
 'sp',
 'flux(V)',
 'flux_error(V)',
 'flux(G)',
 'flux_error(G)']

Do Simbad query:
- solar-type stars (F/G/K)
- plx > plx_min from above
- excluding double stars (and the corresponding sub-categories, see: http://simbad.u-strasbg.fr/simbad/sim-display?data=otypes)

In [439]:
result = customSimbad.query_criteria("(spstring ~ 'F' | spstring ~ 'G' | spstring ~ 'K') & \
                                      (plx >= 50) & \
                                      (maintypes != '**')")

In [440]:
print result.colnames

['MAIN_ID', 'RA', 'DEC', 'RA_PREC', 'DEC_PREC', 'COO_ERR_MAJA', 'COO_ERR_MINA', 'COO_ERR_ANGLE', 'COO_QUAL', 'COO_WAVELENGTH', 'COO_BIBCODE', 'ID_gj', 'ID_hd', 'ID_wds', 'PLX_VALUE', 'PLX_ERROR', 'SP_TYPE', 'FLUX_V', 'FLUX_ERROR_V', 'FLUX_G', 'FLUX_ERROR_G']


## Change empty values to NULL

Some columns can be empty, so I want to have NULL values in those columns:
- ID_hd (empty string, not a masked value)
- ID_gj (same as ID_hd)
- FLUX_ERROR (a masked value)
- FLUX (a masked value)

For string columns it's easy to rename the empty strings:

In [441]:
result[:]['ID_hd'][np.where(result[:]['ID_hd'] == '')] = 'NULL'
result[:]['ID_gj'][np.where(result[:]['ID_gj'] == '')] = 'NULL'
result[:]['ID_wds'][np.where(result[:]['ID_wds'] == '')] = 'NULL'
result[:]['MAIN_ID'][np.where(result[:]['MAIN_ID'] == '')] = 'NULL'

In [442]:
print type(result['ID_wds'][0])

<type 'str'>


More difficult for the flux values... will make a new string-type array for these.

In [443]:
flux_table = np.column_stack( ( np.array(result['FLUX_V'],dtype=str),
                                np.array(result['FLUX_ERROR_V'],dtype=str),
                                np.array(result['FLUX_G'],dtype=str),
                                np.array(result['FLUX_ERROR_G'],dtype=str),
                              )
                            )

In [444]:
flux_table[np.where(flux_table == 'nan')] = 'NULL'

In [445]:
print flux_table

[['6.03000020981' 'NULL' '5.4236998558' '0.00150000001304']
 ['5.21000003815' 'NULL' '4.71649980545' '0.00260000000708']
 ['9.67399978638' '0.019999999553' '9.03890037537' '0.000399999989895']
 ..., 
 ['10.8870000839' 'NULL' '10.0215997696' '0.000600000028498']
 ['11.2559995651' 'NULL' '10.2636003494' '0.0010000000475']
 ['10.3240003586' 'NULL' '9.3983001709' '0.000699999975041']]


## Add results to stars_simbad table

In [446]:
add_simbad_results = '''
INSERT INTO stars_simbad (main_id, hd, gj, wds, ra, dec, plx, plx_err, sptype, V_mag, V_mag_err, G_mag, G_mag_err) 
VALUES('{:s}',{:s},{:s},{:s},'{:s}','{:s}',{:f},{:f},'{:s}',{:s},{:s},{:s},{:s});
'''

In [447]:
null_str_format = '{:s}'
str_format = "'{:s}'"

for r,row in enumerate(result):
    star_id_str = []
    for star_id in [row['ID_hd'],row['ID_gj'],row['ID_wds']]:
        if star_id == 'NULL':
            star_id_str.append(null_str_format.format(star_id))
        else:
            star_id_str.append(str_format.format(star_id))
            
    c.execute(add_simbad_results.format(row['MAIN_ID'],
                                        star_id_str[0],
                                        star_id_str[1],
                                        star_id_str[2],
                                        row['RA'],
                                        row['DEC'],
                                        row['PLX_VALUE'],
                                        row['PLX_ERROR'],
                                        row['SP_TYPE'],
                                        flux_table[r][0],
                                        flux_table[r][1],
                                        flux_table[r][2],
                                        flux_table[r][3],
                                       )
             )

## Save database

In [451]:
connection.commit()

## Perform some SQL queries to check table

In [452]:
c.execute('SELECT * FROM stars_simbad')
rows = c.fetchall()
print len(rows)
print
for row in rows[0:10]:
    print row

452

(1, u'*  61 Cyg B', u'HD 201092', u'GJ 820 B', u'WDS J21069+3845B', u'21 06 55.2640', u'+38 44 31.362', 286.1457, 0.059, u'K7V', 6.03000020981, None, 5.4236998558, 0.00150000001304)
(2, u'*  61 Cyg A', u'HD 201091', u'GJ 820 A', u'WDS J21069+3845A', u'21 06 53.9396', u'+38 44 57.897', 285.9459, 0.1008, u'K5V', 5.21000003815, None, 4.71649980545, 0.00260000000708)
(3, u'BD+40  4631', None, u'GJ 836.8', None, u'21 44 29.9616', u'+41 35 50.862', 50.4206, 0.0281, u'K8V', 9.67399978638, 0.019999999553, 9.03890037537, 0.000399999989895)
(4, u'BD+45  4378', None, u'GJ 913', None, u'23 58 43.4881', u'+46 43 44.737', 58.4067, 0.8984, u'K7V', 9.61999988556, None, 8.87279987335, 0.000500000023749)
(5, u'HD 200560', u'HD 200560', u'GJ 9717 A', u'WDS J21028+4551CD', u'21 02 40.7539', u'+45 53 05.165', 51.3456, 0.0324, u'K2.5V', 7.67999982834, None, 7.4156999588, 0.000300000014249)
(6, u'HD 219134', u'HD 219134', u'GJ 892', u'WDS J23133+5710A', u'23 13 16.9747', u'+57 10 06.076', 153.0808, 0.08

In [453]:
cmd = '''
SELECT main_id,hd,gj,wds FROM stars_simbad
WHERE wds NOT NULL
'''
c.execute(cmd)
rows = c.fetchall()
print len(rows)
print
for row in rows[0:10]:
    print row

221

(u'*  61 Cyg B', u'HD 201092', u'GJ 820 B', u'WDS J21069+3845B')
(u'*  61 Cyg A', u'HD 201091', u'GJ 820 A', u'WDS J21069+3845A')
(u'HD 200560', u'HD 200560', u'GJ 9717 A', u'WDS J21028+4551CD')
(u'HD 219134', u'HD 219134', u'GJ 892', u'WDS J23133+5710A')
(u'* eta Cas B', u'HD 4614B', u'GJ 34 B', u'WDS J00491+5749B')
(u'* tet Per', u'HD 16895', u'GJ 107 A', u'WDS J02442+4914A')
(u'* bet Cas', u'HD 432', u'GJ 8', u'WDS J00092+5909A')
(u'* iot Per', u'HD 19373', u'GJ 124', u'WDS J03091+4937A')
(u'HD  10780', u'HD 10780', u'GJ 75', u'WDS J01477+6351A')
(u'* tet Cyg', u'HD 185395', u'GJ 765 A', u'WDS J19364+5013AB')


In [454]:
cmd = '''
SELECT main_id,hd,gj,wds FROM stars_simbad
WHERE wds NOT NULL AND 
  (hd NOT LIKE '%A' AND hd NOT LIKE '%B' AND hd NOT LIKE '%C' AND hd NOT LIKE '%D') AND
  (gj NOT LIKE '%A' AND gj NOT LIKE '%B' AND hd NOT LIKE '%C' AND hd NOT LIKE '%D')
'''
c.execute(cmd)
rows = c.fetchall()
print len(rows)
print
for row in rows[0:10]:
    print row

92

(u'HD 219134', u'HD 219134', u'GJ 892', u'WDS J23133+5710A')
(u'* bet Cas', u'HD 432', u'GJ 8', u'WDS J00092+5909A')
(u'* iot Per', u'HD 19373', u'GJ 124', u'WDS J03091+4937A')
(u'HD  10780', u'HD 10780', u'GJ 75', u'WDS J01477+6351A')
(u'* eta Cep', u'HD 198149', u'GJ 807', u'WDS J20453+6150A')
(u'HD  13579', u'HD 13579', u'GJ 90', u'WDS J02157+6740A')
(u'V* BY Dra', u'HD 234677', u'GJ 719', u'WDS J18339+5144A')
(u'* sig Dra', u'HD 185144', u'GJ 764', u'WDS J19324+6940A')
(u'HD  37394', u'HD 37394', u'GJ 211', u'WDS J05413+5329A')
(u'HD 153525', u'HD 153525', u'GJ 649.1 C', u'WDS J16579+4722C')


## Close connection when finished

In [4]:
connection.close()